In [1]:
just_checking_integrity=False

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

2024-02-19 18:16:39.051435: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-19 18:16:39.051471: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
items_features = pd.read_csv('../dataset/processed_data/simplified_features_and_categories_30.csv')
items_features.head()

,item_id,feature_idx
0,19021,0
1,19021,22
2,19021,15
3,19021,446
4,19021,1


In [4]:
items_features['item_id'].max()

23691

In [5]:
dataset = np.zeros(
    (items_features['item_id'].max()+1, items_features['feature_idx'].max()+1)
    )
dataset[(items_features['item_id'], items_features['feature_idx'])] = 1

dataset
train_dataset=dataset[1:,:]

In [6]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(train_dataset,test_size=0.2,random_state=941116)

In [7]:
dataset.shape

(23692, 636)

In [8]:
train.shape

(18952, 636)

In [9]:
test.shape

(4739, 636)

# Autoencoder

In [10]:
latent_dimension = 64
starting_dimension = 512

inputs = layers.Input(int(items_features['feature_idx'].max()+1))

x = inputs 
dimension = starting_dimension

while(dimension != latent_dimension):
    #x = layers.BatchNormalization()(x)
    x = layers.Dense(units=dimension, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
    x = layers.Dropout(0.1)(x)

    dimension = dimension//2

x = layers.Dense(units=dimension,activation="tanh",activity_regularizer=tf.keras.regularizers.L1L2(l1=5e-5,l2=4e-4))(x)
outputs = x
encoder = tf.keras.Model(inputs=inputs, outputs=outputs, name='encoder')

2024-02-19 18:16:43.904799: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-02-19 18:16:43.904833: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-02-19 18:16:43.904862: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-0a32aa): /proc/driver/nvidia/version does not exist
2024-02-19 18:16:43.905174: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
max_width = 1024
ending_dimension = items_features['feature_idx'].max()+1

inputs = layers.Input(latent_dimension)

x = inputs 
dimension = latent_dimension

x = layers.Dense(units=256, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(units=512, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
x = layers.Dropout(0.2)(x)
    

x = layers.Dense(units=ending_dimension,activation="sigmoid")(x)
outputs = x

decoder = tf.keras.Model(inputs=inputs, outputs=outputs, name='decoder')

#decoder.summary()

In [12]:
ae = tf.keras.Model(inputs=encoder.inputs, outputs=decoder(encoder.outputs), name='auto_encoder')
ae.summary()

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 636)]             0         
                                                                 
 dense (Dense)               (None, 512)               326144    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)              

In [13]:
def custom_loss(x_true,x_pred):
    return tf.keras.losses.cosine_similarity(x_true,x_pred)+tf.keras.losses.mean_squared_error(x_true,x_pred)#

In [14]:
ae.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.006), 
    loss=custom_loss,
    metrics=[tf.keras.metrics.CosineSimilarity(),tf.keras.metrics.MeanAbsoluteError()]
    )

# Training

In [15]:
ae.fit(
    train,
    train, 
    batch_size=64,
    validation_data=(test,test),
    epochs=5 if just_checking_integrity else 200,
    shuffle=True,
    callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='cosine_similarity',
                patience=10,
                restore_best_weights=True,
                mode="max"
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_cosine_similarity',
                factor=0.7,
                patience=2,
                min_lr=5e-5,
                mode="max",
                min_delta=0.0001,
            )
        ]
    )

2024-02-19 18:16:44.647966: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 48213888 exceeds 10% of free system memory.
2024-02-19 18:16:44.755237: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 48213888 exceeds 10% of free system memory.


Epoch 1/200
297/297 [==============================] - 6s 13ms/step - loss: -0.7641 - cosine_similarity: 0.8002 - mean_absolute_error: 0.0484 - val_loss: -0.8530 - val_cosine_similarity: 0.8767 - val_mean_absolute_error: 0.0300 - lr: 0.0060
Epoch 2/200
297/297 [==============================] - 3s 12ms/step - loss: -0.8570 - cosine_similarity: 0.8796 - mean_absolute_error: 0.0294 - val_loss: -0.8758 - val_cosine_similarity: 0.8960 - val_mean_absolute_error: 0.0252 - lr: 0.0060
Epoch 3/200
297/297 [==============================] - 3s 12ms/step - loss: -0.8747 - cosine_similarity: 0.8948 - mean_absolute_error: 0.0260 - val_loss: -0.8924 - val_cosine_similarity: 0.9107 - val_mean_absolute_error: 0.0223 - lr: 0.0060
Epoch 4/200
297/297 [==============================] - 3s 11ms/step - loss: -0.8887 - cosine_similarity: 0.9072 - mean_absolute_error: 0.0232 - val_loss: -0.9044 - val_cosine_similarity: 0.9211 - val_mean_absolute_error: 0.0204 - lr: 0.0060
Epoch 5/200
297/297 [===============

In [16]:
temp=encoder(dataset)

2024-02-19 18:24:42.439812: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 120544896 exceeds 10% of free system memory.
2024-02-19 18:24:42.489187: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60272448 exceeds 10% of free system memory.
2024-02-19 18:24:42.531448: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 48521216 exceeds 10% of free system memory.


In [17]:
temp.shape

TensorShape([23692, 64])

In [18]:
temp

<tf.Tensor: shape=(23692, 64), dtype=float32, numpy=
array([[-0.02569672, -0.04465982,  0.04233517, ...,  0.02195858,
         0.00406342,  0.00727194],
       [ 0.01188441,  0.09571718,  0.27183083, ..., -0.05592808,
        -0.1315058 ,  0.10294587],
       [-0.04093751,  0.01779453,  0.09653322, ...,  0.00493379,
        -0.05013884,  0.00424234],
       ...,
       [-0.00145441, -0.09328362,  0.14141937, ..., -0.05779222,
        -0.07302618, -0.01466742],
       [ 0.03092937,  0.1346339 , -0.02694407, ..., -0.10221646,
        -0.02603671, -0.11175551],
       [ 0.00837145,  0.16141906,  0.01974098, ..., -0.00808643,
        -0.0482159 , -0.11793464]], dtype=float32)>

In [19]:
tf.reduce_sum(temp,axis=1).numpy().max(),tf.reduce_sum(temp,axis=1).numpy().min(),tf.reduce_sum(temp,axis=1).numpy().std()

(4.421576, -4.1391034, 1.0844337)

In [20]:
np.save('../dataset/processed_data/compressed_features', encoder(dataset))